In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS,DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

In [2]:
load_dotenv()
API_KEY = os.getenv('OPENAI_API_KEY')
Model = 'gpt-3.5-turbo'


In [3]:
model = ChatOpenAI(api_key=API_KEY,model=Model)
model.invoke('what is coding?')

AIMessage(content='Coding is the process of creating instructions for a computer to follow in order to perform specific tasks or functions. This is typically done using a programming language, which is a set of rules and syntax that allows developers to communicate with computers and create software, websites, apps, and more. Coding involves writing lines of code that tell the computer how to execute certain actions, manipulate data, and produce desired outcomes. It requires logical thinking, problem-solving skills, and attention to detail.', response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 11, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-bdfad552-8a8c-4771-8b65-8b30db5bd9d0-0')

In [5]:
parser = StrOutputParser()
chain = model|parser

In [6]:
chain.invoke('what is coding?')

'Coding is the process of using a programming language to create instructions that tell a computer how to perform a specific task or function. It involves writing lines of code, which are commands and algorithms that are written in a specific syntax to create software, websites, applications, or other technological solutions. Coding is used by programmers, developers, and software engineers to create digital products and services that power the modern world.'

In [8]:
file_loader = PyPDFLoader('malnutrition detection.pdf')
page = file_loader.load_and_split()
len(page)

7

In [27]:
spliter = RecursiveCharacterTextSplitter(chunk_size = 200,chunk_overlap = 50)
pages = spliter.split_documents(page)
pages[3]

Document(page_content='of Science and Technology,  \nChennai, India.  \nsaravanan_cse_2019@crescent.educati\nonPavani B  \nDepartment of Computer Science and \nEngineering,  \nB. S. Abdur Rahman Crescent Institute', metadata={'source': 'malnutrition detection.pdf', 'page': 0})

In [12]:
vector_storage = FAISS.from_documents(pages,OpenAIEmbeddings())
retriever = vector_storage.as_retriever()

In [14]:
question_template = """
your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context:{context}
question:{question}

"""

In [17]:
prompt = PromptTemplate.from_template(template=question_template)
print(prompt.format(context = ' Here is the context to use',
              question = ' Answer this question based on the context'
              ))


your a smart bot that answers questions based on the context given to you only.
You don't make things up.
context: Here is the context to use
question: Answer this question based on the context




In [18]:
result = RunnableParallel(context= retriever,question = RunnablePassthrough())

In [19]:
chain = result |prompt | model | parser

In [28]:
chain.invoke('how many classes does the model classifies the input image into?')

'The model classifies the input image into 3 classes.'

In [29]:
retriever.invoke('how many classes does the model classifies the input image into?')

[Document(page_content='D. Classification \nThis module classifies the input image into either', metadata={'source': 'malnutrition detection.pdf', 'page': 2}),
 Document(page_content='Then, it trains the entire network to classify the images, here', metadata={'source': 'malnutrition detection.pdf', 'page': 2}),
 Document(page_content='results were classified into 3 categories based upon image', metadata={'source': 'malnutrition detection.pdf', 'page': 3}),
 Document(page_content='Classification using image result and parametric conditions', metadata={'source': 'malnutrition detection.pdf', 'page': 1})]